In [1]:
import cv2
import torch
import warnings
import numpy as np
import random
from sklearn.linear_model import LinearRegression
import time
from flask import Flask, render_template_string, jsonify, send_file
from threading import Thread
import os

# Suppress all Future Warning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialize Flask app
app = Flask(__name__)

# Linear Regression Model
data = np.array([
    [0, 0],
    [2, 4],
    [5, 7],
    [10, 14],
    [15, 17],
    [20, 23],
    [25, 28],
    [30, 33],
    [35, 35],
])
X = data[:, 0].reshape(-1, 1)
y = data[:, 1]

model_L = LinearRegression()
model_L.fit(X, y)

# Load a larger YOLOv5 model for better detection
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# Set constants
GREEN_LIGHT_THRESHOLD = 35 
YELLOW_LIGHT_DURATION = 3    

# Initialize lane status, current image, and timers
lane_status = ["red", "red", "red", "red"]
current_image_path = ""
lane_timers = [0, 0, 0, 0]

# Function to count vehicles in a single lane using YOLO
def count_vehicles(frame):
    results = model(frame)
    detections = results.pandas().xyxy[0]
    vehicle_classes = ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'scooty']
    
    # Filter by vehicle class and confidence threshold
    detections = detections[(detections['name'].isin(vehicle_classes)) & (detections['confidence'] > 0.3)]
    
    return detections.shape[0]

def predict_time(vehicle_count):
    return model_L.predict(np.array([[vehicle_count]]))[0]

def calculate_clearance_time(vehicle_count):
    clearance_time = min(predict_time(vehicle_count), GREEN_LIGHT_THRESHOLD)
    return clearance_time

# Function to process lanes
def process_lanes():
    global lane_status, current_image_path, lane_timers
    total_time = 0
    for lane_index in range(4):
        lane_status = ["red"] * 4  # Reset all lanes to red
        lane_timers = [0] * 4  # Reset all timers
        
        # Simulate green light for the current lane
        lane_status[lane_index] = "green"
        
        # Generate a random image path
        current_image_path = generate_random_image_path()
        frame = cv2.imread(current_image_path)

        # Process the frame to count vehicles
        vehicle_count = count_vehicles(frame)
        clearance_time = calculate_clearance_time(vehicle_count)
        lane_timers[lane_index] = int(clearance_time)

        print(f"Lane {lane_index+1}: {clearance_time:.2f} sec green + {YELLOW_LIGHT_DURATION} sec yellow vehicles {vehicle_count}")
        update_lane_status()  # Update the web display
        
        while lane_timers[lane_index] > 0:
            time.sleep(1)
            lane_timers[lane_index] -= 1
            update_lane_status()  # Update the web display

        # Simulate yellow light
        lane_status[lane_index] = "yellow"
        print(f"Lane {lane_index+1} yellow light on.")
        update_lane_status()  # Update the web display
        time.sleep(YELLOW_LIGHT_DURATION)

        lane_status[lane_index] = "red"
        print(f"Lane {lane_index+1} red light on.")
        update_lane_status()  # Update the web display
        total_time += clearance_time + YELLOW_LIGHT_DURATION
    print("Completed one full cycle of lane processing.\n")
    print(f"Total time for this iteration: {total_time:.2f} seconds\n")

# Function to generate a random image path dynamically
def generate_random_image_path():
    path = "D:/SIH/Data/img/"
    random_number = random.randint(1, 57)
    image_path = f"{path}traffic_img({random_number}).jpg"
    return image_path

# Update lane status function
def update_lane_status():
    with app.app_context():
        return lane_status

# Flask route to serve the lane status
@app.route('/lane_status')
def get_lane_status():
    return jsonify({
        "status": lane_status,
        "timers": lane_timers
    })

# Flask route to serve the current image
@app.route('/current_image')
def get_current_image():
    if current_image_path and os.path.exists(current_image_path):
        return send_file(current_image_path, mimetype='image/jpeg')
    else:
        return "", 404

# Integrated HTML template with Flask
@app.route('/')
def index():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Traffic Signal Simulation</title>
            <style>
                .lane-container {
                    display: inline-block;
                    text-align: center;
                    margin: 20px;
                }
                .lane {
                    width: 100px;
                    height: 100px;
                    border: 2px solid black;
                    line-height: 100px;
                    font-size: 20px;
                    color: white;
                    font-weight: bold;
                    border-radius: 50%;
                    position: relative;
                }
                .red { background-color: red; }
                .yellow { background-color: yellow; color: black; }
                .green { background-color: green; }
                .timer {
                    font-size: 18px;
                    color: black;
                    font-weight: bold;
                    margin-bottom: 10px;
                }
                .traffic-image {
                    width: 400px;
                    height: 300px;
                    margin: 20px;
                    border: 2px solid black;
                    display: block;
                }
            </style>
        </head>
        <body>
            <h1>Traffic Signal Simulation</h1>
            <div class="lane-container">
                <div class="timer" id="timer1"></div>
                <div class="lane red" id="lane1">Lane 1</div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer2"></div>
                <div class="lane red" id="lane2">Lane 2</div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer3"></div>
                <div class="lane red" id="lane3">Lane 3</div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer4"></div>
                <div class="lane red" id="lane4">Lane 4</div>
            </div>
            <img src="/current_image" id="traffic-image" class="traffic-image" alt="Traffic Image">

            <script>
                function updateLaneColors() {
                    fetch('/lane_status')
                        .then(response => response.json())
                        .then(data => {
                            const lanes = data.status;
                            const timers = data.timers;
                            document.getElementById('lane1').className = 'lane ' + lanes[0];
                            document.getElementById('lane2').className = 'lane ' + lanes[1];
                            document.getElementById('lane3').className = 'lane ' + lanes[2];
                            document.getElementById('lane4').className = 'lane ' + lanes[3];
                            document.getElementById('timer1').textContent = timers[0] > 0 ? timers[0] + 's' : '';
                            document.getElementById('timer2').textContent = timers[1] > 0 ? timers[1] + 's' : '';
                            document.getElementById('timer3').textContent = timers[2] > 0 ? timers[2] + 's' : '';
                            document.getElementById('timer4').textContent = timers[3] > 0 ? timers[3] + 's' : '';
                        });
                }

                function updateTrafficImage() {
                    document.getElementById('traffic-image').src = '/current_image?' + new Date().getTime();
                }

                // Update lane colors, timers, and traffic image every second
                setInterval(updateLaneColors, 1000);
                setInterval(updateTrafficImage, 1000);
            </script>
        </body>
        </html>
    ''')

if __name__ == '__main__':
    # Start Flask app in a separate thread
    thread = Thread(target=lambda: app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False))
    thread.start()

    # Run the lane processing loop
    while True:
        process_lanes()


Using cache found in C:\Users\samki/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-22 Python-3.12.5 torch-2.4.0+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.110.26:5000
Press CTRL+C to quit


Lane 1: 4.01 sec green + 3 sec yellow vehicles 2
Lane 1 yellow light on.
Lane 1 red light on.
Lane 2: 10.05 sec green + 3 sec yellow vehicles 8


192.168.110.26 - - [14/Sep/2024 13:02:41] "GET / HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:41] "GET /current_image HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:42] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:42] "GET /current_image?1726299162504 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:43] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:43] "GET /current_image?1726299163506 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:44] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:02:44] "GET /current_image?1726299164498 HTTP/1.1" 200 -


...

192.168.110.26 - - [14/Sep/2024 13:06:34] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:34] "GET /current_image?1726299394075 HTTP/1.1" 200 -


Lane 1: 31.21 sec green + 3 sec yellow vehicles 29


192.168.110.26 - - [14/Sep/2024 13:06:35] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:35] "GET /current_image?1726299395074 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:36] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:36] "GET /current_image?1726299396076 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:37] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:37] "GET /current_image?1726299397075 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:38] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:06:38] "GET /current_image?1726299398077 HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:09:17] "GET /lane_status HTTP/1.1" 200 -
192.168.110.26 - - [14/Sep/2024 13:09:17] "GET /current_image?1726299556661 HTTP/1.1" 200 -


Lane 1 yellow light on.
Lane 1 red light on.
Lane 2: 13.08 sec green + 3 sec yellow vehicles 11
Lane 2 yellow light on.
Lane 2 red light on.
Lane 3: 5.02 sec green + 3 sec yellow vehicles 3
Lane 3 yellow light on.
Lane 3 red light on.
Lane 4: 28.19 sec green + 3 sec yellow vehicles 26
Lane 4 yellow light on.
Lane 4 red light on.
Completed one full cycle of lane processing.

Total time for this iteration: 89.49 seconds

Lane 1: 16.10 sec green + 3 sec yellow vehicles 14
Lane 1 yellow light on.
Lane 1 red light on.
Lane 2: 31.21 sec green + 3 sec yellow vehicles 29


KeyboardInterrupt: 

In [1]:
import cv2
import torch
import warnings
import numpy as np
import random
from sklearn.linear_model import LinearRegression
import time
from flask import Flask, render_template_string, jsonify, send_file
from threading import Thread
import os

# Suppress all Future Warning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialize Flask app
app = Flask(__name__)

# Linear Regression Model
data = np.array([
    [0, 0],
    [2, 4],
    [5, 7],
    [10, 14],
    [15, 17],
    [20, 23],
    [25, 28],
    [30, 33],
    [35, 35],
])
X = data[:, 0].reshape(-1, 1)
y = data[:, 1]

model_L = LinearRegression()
model_L.fit(X, y)

# Load a larger YOLOv5 model for better detection
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

# Set constants
GREEN_LIGHT_THRESHOLD = 35 
YELLOW_LIGHT_DURATION = 3    

# Initialize lane status, current image, and timers
lane_status = ["red", "red", "red", "red"]
current_image_paths = ["", "", "", ""]
lane_timers = [0, 0, 0, 0]
vehicle_counts = [0, 0, 0, 0]

# Image paths for each lane
path = "F:/SIH/Data/img/"
image_paths = [
    path+"1.jpeg", path+"2.jpeg", path+"3.jpeg", path+"4.jpeg",
    
]

# Organize into tuples of 4 images each
lane_image_sets = [tuple(image_paths[i:i+4]) for i in range(0, len(image_paths), 4)]

# Initialize a global counter for image sets
current_image_set_index = 0

# Function to count vehicles in a single lane using YOLO
def count_vehicles(frame):
    results = model(frame)
    detections = results.pandas().xyxy[0]
    vehicle_classes = ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'scooty']
    
    # Filter by vehicle class and confidence threshold
    detections = detections[(detections['name'].isin(vehicle_classes)) & (detections['confidence'] > 0.3)]
    
    return detections.shape[0]

def predict_time(vehicle_count):
    return model_L.predict(np.array([[vehicle_count]]))[0]

def calculate_clearance_time(vehicle_count):
    clearance_time = min(predict_time(vehicle_count), GREEN_LIGHT_THRESHOLD)
    return clearance_time

# Update lane status function (placeholder)
def update_lane_status():
    pass  # Flask will handle UI updates by fetching data

# Function to process lanes
def process_lanes():
    global lane_status, current_image_paths, lane_timers, vehicle_counts, current_image_set_index
    total_time = 0

    # Loop through each lane to process
    for lane_index in range(len(lane_status)):  # Iterate over lanes
        # Reset all lanes to red
        lane_status = ["red"] * 4
        lane_timers = [0] * 4
        vehicle_counts = [0] * 4

        # Simulate green light for the current lane
        lane_status[lane_index] = "green"

        # Use the current set of images, rotating through all available sets
        current_image_paths = lane_image_sets[current_image_set_index]

        # Load the image for the current lane
        frame = cv2.imread(current_image_paths[lane_index])

        # Process the frame to count vehicles
        vehicle_count = count_vehicles(frame)
        vehicle_counts[lane_index] = vehicle_count  # Store the vehicle count
        clearance_time = calculate_clearance_time(vehicle_count)
        lane_timers[lane_index] = int(clearance_time)

        print(f"Lane {lane_index + 1}: {clearance_time:.2f} sec green + {YELLOW_LIGHT_DURATION} sec yellow | Vehicles: {vehicle_count}")
        update_lane_status()  # Update the web display

        # Wait for the green light duration to complete
        while lane_timers[lane_index] > 0:
            time.sleep(1)
            lane_timers[lane_index] -= 1
            update_lane_status()  # Update the web display

        # Simulate yellow light
        lane_status[lane_index] = "yellow"
        print(f"Lane {lane_index + 1} yellow light on.")
        update_lane_status()  # Update the web display
        time.sleep(YELLOW_LIGHT_DURATION)

        lane_status[lane_index] = "red"
        print(f"Lane {lane_index + 1} red light on.")
        update_lane_status()  # Update the web display
        total_time += clearance_time + YELLOW_LIGHT_DURATION

    # Move to the next set of images for the next iteration
    current_image_set_index = (current_image_set_index + 1) % len(lane_image_sets)

    print("Completed one full cycle of lane processing.\n")
    print(f"Total time for this iteration: {total_time:.2f} seconds\n")

# Flask route to serve the lane status
@app.route('/lane_status')
def get_lane_status():
    return jsonify({
        "status": lane_status,
        "timers": lane_timers,
        "vehicle_counts": vehicle_counts
    })

# Flask route to serve the current image for a specific lane
@app.route('/current_image/<int:lane>')
def get_current_image(lane):
    if current_image_paths and os.path.exists(current_image_paths[lane]):
        return send_file(current_image_paths[lane], mimetype='image/jpeg')
    else:
        return "", 404

# Integrated HTML template with Flask
@app.route('/')
def index():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Traffic Signal Simulation</title>
            <style>
                .lane-container {
                    display: inline-block;
                    text-align: center;
                    margin: 20px;
                }
                .lane {
                    width: 100px;
                    height: 100px;
                    border: 2px solid black;
                    line-height: 100px;
                    font-size: 20px;
                    color: white;
                    font-weight: bold;
                    border-radius: 50%;
                    position: relative;
                }
                .red { background-color: red; }
                .yellow { background-color: yellow; color: black; }
                .green { background-color: green; }
                .timer {
                    font-size: 18px;
                    color: black;
                    font-weight: bold;
                    margin-bottom: 10px;
                }
                .traffic-image {
                    width: 400px;
                    height: 300px;
                    margin: 20px;
                    border: 2px solid black;
                    display: block;
                }
            </style>
        </head>
        <body>
            <h1>Traffic Signal Simulation</h1>
            <div class="lane-container">
                <div class="timer" id="timer1"></div>
                <div class="lane red" id="lane1">Lane 1</div>
                <div id="vehicle_count1" class="timer"></div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer2"></div>
                <div class="lane red" id="lane2">Lane 2</div>
                <div id="vehicle_count2" class="timer"></div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer3"></div>
                <div class="lane red" id="lane3">Lane 3</div>
                <div id="vehicle_count3" class="timer"></div>
            </div>
            <div class="lane-container">
                <div class="timer" id="timer4"></div>
                <div class="lane red" id="lane4">Lane 4</div>
                <div id="vehicle_count4" class="timer"></div>
            </div>
            <img src="/current_image/0" id="traffic-image" class="traffic-image" alt="Traffic Image">

            <script>
                function updateLaneColors() {
                    fetch('/lane_status')
                        .then(response => response.json())
                        .then(data => {
                            const lanes = data.status;
                            const timers = data.timers;
                            const vehicleCounts = data.vehicle_counts;
                            document.getElementById('lane1').className = 'lane ' + lanes[0];
                            document.getElementById('lane2').className = 'lane ' + lanes[1];
                            document.getElementById('lane3').className = 'lane ' + lanes[2];
                            document.getElementById('lane4').className = 'lane ' + lanes[3];
                            document.getElementById('timer1').innerText = timers[0] + ' sec';
                            document.getElementById('timer2').innerText = timers[1] + ' sec';
                            document.getElementById('timer3').innerText = timers[2] + ' sec';
                            document.getElementById('timer4').innerText = timers[3] + ' sec';
                            document.getElementById('vehicle_count1').innerText = 'Vehicles: ' + vehicleCounts[0];
                            document.getElementById('vehicle_count2').innerText = 'Vehicles: ' + vehicleCounts[1];
                            document.getElementById('vehicle_count3').innerText = 'Vehicles: ' + vehicleCounts[2];
                            document.getElementById('vehicle_count4').innerText = 'Vehicles: ' + vehicleCounts[3];

                            // Update the traffic image based on the current lane status
                            const activeLane = lanes.indexOf('green');
                            if (activeLane !== -1) {
                                document.getElementById('traffic-image').src = '/current_image/' + activeLane + '?v=' + new Date().getTime();
                            }
                        });
                }

                setInterval(updateLaneColors, 1000);  // Update every second
            </script>
        </body>
        </html>
    ''')

# Start the lane processing in a separate thread
thread_flask = Thread(target=lambda: app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False))
thread_flask.start()

# Start the lane processing loop
while True:
    process_lanes()
    time.sleep(1)


Using cache found in C:\Users\samki/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-22 Python-3.12.5 torch-2.4.0+cpu CPU

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.8:5000
Press CTRL+C to quit
192.168.1.8 - - [15/Sep/2024 13:03:08] "GET / HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:08] "GET /current_image/0 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:09] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:09] "GET /current_image/0?v=1726385589856 HTTP/1.1" 200 -


Lane 1: 23.15 sec green + 3 sec yellow | Vehicles: 21


192.168.1.8 - - [15/Sep/2024 13:03:10] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:10] "GET /current_image/0?v=1726385590865 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:11] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:11] "GET /current_image/0?v=1726385591891 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:12] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:12] "GET /current_image/0?v=1726385592863 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:13] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:13] "GET /current_image/0?v=1726385593855 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:14] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:14] "GET /current_image/0?v=1726385594855 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:15] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:03:15] "GET /current_image/0?v=1726385595851 HTTP/1.1" 200 -
192.

...

192.168.1.8 - - [15/Sep/2024 13:05:01] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:01] "GET /current_image/2?v=1726385701085 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:02] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:02] "GET /current_image/2?v=1726385702087 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:03] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:03] "GET /current_image/2?v=1726385703082 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:04] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:04] "GET /current_image/2?v=1726385704085 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:05] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:05] "GET /current_image/2?v=1726385705090 HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:06] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:06] "GET /current_image/2?v=1726385706084 HTTP/1.1" 200 -
192.

Lane 3 yellow light on.


192.168.1.8 - - [15/Sep/2024 13:05:16] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:17] "GET /lane_status HTTP/1.1" 200 -
192.168.1.8 - - [15/Sep/2024 13:05:18] "GET /lane_status HTTP/1.1" 200 -


KeyboardInterrupt: 

192.168.1.8 - - [15/Sep/2024 13:05:18] "GET /lane_status HTTP/1.1" 200 -
